In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('Results/RequiredDataSet.csv',header=None, names=['booking_date','channel','property_id','room_name','room_id','check_in','check_out','room_rate','booking_status'])
# data.head()


In [2]:
Results = pd.read_excel('Results/DataSetWithStayDate.xlsx')
del Results['Old_Room_rate']
# Results.head()

In [3]:
scrapyData = pd.read_csv('Results/eventData2015.csv',header=None, names=['Day','Date','Event','HolidayIn'])
scrapeResult = scrapyData.dropna()
# scrapeResult.head()

In [4]:
# to avoid "SettingWithCopyWarning"
pd.options.mode.chained_assignment = None  # default='warn'

In [5]:
scrapeResult['Date'] =   '2015-' + scrapeResult['Date'] 
# scrapeResult.head()

In [6]:
scrapeResult['Date'] = pd.to_datetime(scrapeResult.Date)
# scrapeResult.head()

In [7]:
del scrapeResult['HolidayIn']
del scrapeResult['Day']
# scrapeResult.head()

In [8]:
newResults = Results.filter(['Property_id','Stay_date','Booking_status','Room_rate'], axis=1)
# newResults.head()

In [9]:
newResults = newResults[(newResults.Booking_status == 'COMPLETED') | (newResults.Booking_status == 'CONFIRMED')]
# newResults

In [10]:
newResults = newResults[(newResults.Stay_date >= '2015-01-01') & (newResults.Stay_date <= '2015-12-31')]
# newResults.head()

In [11]:
newResults['Stay_date'] = pd.to_datetime(newResults.Stay_date)
# newResults.head()

In [12]:
res = pd.merge(left=newResults, left_on='Stay_date',right=scrapeResult, right_on='Date')
# res1 = res.drop_duplicates()
res1 = res.reset_index()
del res1['index']
del res1['Date']
# res1.head()

In [13]:
res1['room_count'] = res1.groupby(['Property_id','Stay_date'])['Room_rate'].transform('count')
res2 = res1.drop_duplicates()
res2.head()

,Property_id,Stay_date,Booking_status,Room_rate,Event,room_count
0,BB,2015-01-01,COMPLETED,39500.00,New Years Day,26
2,KR,2015-01-01,COMPLETED,19387.25,New Years Day,12
3,SS,2015-01-01,COMPLETED,20130.00,New Years Day,23
4,KK,2015-01-01,COMPLETED,447.35,New Years Day,13
5,SS,2015-01-01,COMPLETED,31008.66,New Years Day,23


In [14]:
res2['totalRevenue'] = res2.groupby(['Property_id','Stay_date'])['Room_rate'].transform('sum')
res3 = res2.drop_duplicates()
res3.head()

,Property_id,Stay_date,Booking_status,Room_rate,Event,room_count,totalRevenue
0,BB,2015-01-01,COMPLETED,39500.00,New Years Day,26,226229.66
2,KR,2015-01-01,COMPLETED,19387.25,New Years Day,12,170272.65
3,SS,2015-01-01,COMPLETED,20130.00,New Years Day,23,293631.32
4,KK,2015-01-01,COMPLETED,447.35,New Years Day,13,53759.07
5,SS,2015-01-01,COMPLETED,31008.66,New Years Day,23,293631.32


In [15]:
res3['ADR'] = res3.apply(lambda res3: (res3['totalRevenue'] / res3['room_count'] ) , axis=1)
res4 =res3
res4.head()

,Property_id,Stay_date,Booking_status,Room_rate,Event,room_count,totalRevenue,ADR
0,BB,2015-01-01,COMPLETED,39500.00,New Years Day,26,226229.66,8701.140769
2,KR,2015-01-01,COMPLETED,19387.25,New Years Day,12,170272.65,14189.387500
3,SS,2015-01-01,COMPLETED,20130.00,New Years Day,23,293631.32,12766.579130
4,KK,2015-01-01,COMPLETED,447.35,New Years Day,13,53759.07,4135.313077
5,SS,2015-01-01,COMPLETED,31008.66,New Years Day,23,293631.32,12766.579130


In [16]:
# len(res4[res4.channel=="OTA"])
del res4['Booking_status']
del res4['Room_rate']
res5 = res4.drop_duplicates()
# res5

In [17]:
res5.head()

,Property_id,Stay_date,Event,room_count,totalRevenue,ADR
0,BB,2015-01-01,New Years Day,26,226229.660000,8701.140769
2,KR,2015-01-01,New Years Day,12,170272.650000,14189.387500
3,SS,2015-01-01,New Years Day,23,293631.320000,12766.579130
4,KK,2015-01-01,New Years Day,13,53759.070000,4135.313077
9,MB,2015-01-01,New Years Day,62,422873.764848,6820.544594


In [29]:
res5 = res5.sort_values(by = 'Stay_date', ascending=True)

In [30]:
res5.Property_id.unique()

array(['BB', 'MP', 'SC', 'VM', 'EB', 'SV', 'CL', 'MM', 'BG', 'MB', 'KK',
       'SS', 'KR', 'CH', 'CP', 'VV', 'GF', 'AA'], dtype=object)

In [39]:
property_id = res5[res5.Property_id == input('Enter a property_id: ')]

Enter a property_id: MP


In [47]:
#graph plotting

In [41]:
# Our numerical workhorses
import numpy as np
import pandas as pd


# Import Bokeh modules for interactive plotting
import bokeh.charts
import bokeh.charts.utils
import bokeh.io
import bokeh.models
import bokeh.palettes
import bokeh.plotting

# Display graphics in this notebook
bokeh.io.output_notebook()

Loading BokehJS ...

In [42]:
import bokeh.plotting as plt
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.models.formatters import DatetimeTickFormatter
from bokeh.models import HoverTool,WheelZoomTool,ResetTool
# from bokeh.io import output_notebook

source = ColumnDataSource(
                                data=dict(
                                            x = property_id['Stay_date'],
                                            y = property_id['room_count'],
                                            date = [x.strftime("%Y-%m-%d") for x in property_id['Stay_date']],
                                            room_count = property_id['room_count'],
                                            event = property_id['Event']
                                          )
                             )



hover = HoverTool(
                    tooltips=[
                                ("Date", "@date"),
                                ("Room Counts", "@room_count"),
#                                 ("Total revenue", "@y{1.11}"),
                                ("Event", "@event"),
                             ]
                 )

p = plt.figure(x_axis_type = "datetime", x_axis_label = "Date", y_axis_label = "Occupancy", width=800, height=500, tools=[hover,WheelZoomTool(),ResetTool()], toolbar_location="right", title="Stay Date vs Occupancy")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source,legend = "Occupancy" )   

show(p)

In [43]:
import bokeh.plotting as plt
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.models.formatters import DatetimeTickFormatter
from bokeh.models import HoverTool,WheelZoomTool,ResetTool
# from bokeh.io import output_notebook

source = ColumnDataSource(
                                data=dict(
                                            x = property_id['Stay_date'],
                                            y = property_id['totalRevenue'],
                                            date = [x.strftime("%Y-%m-%d") for x in property_id['Stay_date']],
                                            room_count = property_id['room_count'],
                                            event = property_id['Event']
                                          )
                             )



hover = HoverTool(
                    tooltips=[
                                ("Date", "@date"),
                                ("Room Counts", "@room_count"),
                                ("Total revenue", "@y{1.11}"),
                                ("Event", "@event"),
                             ]
                 )

p = plt.figure(x_axis_type = "datetime", x_axis_label = "Date", y_axis_label = "Total Revenue", width=800, height=500, tools=[hover,WheelZoomTool(),ResetTool()], toolbar_location="right", title="Stay Date vs Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source,legend = "Total Revenue" )   

show(p)

In [44]:
import bokeh.plotting as plt
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.models.formatters import DatetimeTickFormatter
from bokeh.models import HoverTool,WheelZoomTool,ResetTool
# from bokeh.io import output_notebook

source = ColumnDataSource(
                                data=dict(
                                            x = property_id['Stay_date'],
                                            y = property_id['ADR'],
                                            date = [x.strftime("%Y-%m-%d") for x in property_id['Stay_date']],
                                            room_count = property_id['room_count'],
                                            event = property_id['Event']
                                          )
                             )



hover = HoverTool(
                    tooltips=[
                                ("Date", "@date"),
                                ("Room Counts", "@room_count"),
                                ("ADR", "@y{1.11}"),
                                ("Event", "@event"),
                             ]
                 )

p = plt.figure(x_axis_type = "datetime", x_axis_label = "Date", y_axis_label = "ADR", width=800, height=500, tools=[hover,WheelZoomTool(),ResetTool()], toolbar_location="right", title="Stay Date vs ADR")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source,legend = "ADR" )   

show(p)

In [45]:
TotalRevenue = property_id["totalRevenue"].mean()

roomcount = property_id["room_count"].sum()

adr = property_id["ADR"].mean()

totalRoomCount = property_id["room_count"].count()

In [46]:
print("Total Room Count : "+str(totalRoomCount))
print("\n")

# print("Sum of Room Count : "+str(roomcount))
# print("\n")

print("ADR : "+str(adr))
print("\n")

print("Average Total Revenue : "+str(TotalRevenue))

Total Room Count : 77


ADR : 3084.5448804370244


Average Total Revenue : 18484.623246753246
